In [39]:
#네이버 뉴스 크롤링

import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd


client_id = "0KKFh6Cii69eEp1dWM73"
client_secret = "GgCVRePonV"


#[CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

#[CODE 2]
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   #[CODE 1]
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

#[CODE 3]
def getPostData(post, jsonResult, cnt):    
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    result = []

    
    pDate = datetime.datetime.strptime(post['pubDate'],  '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 
'org_link':org_link,   'link': org_link,   'pDate':pDate})
    result.append({cnt, title, description, org_link, pDate})

    return

#[CODE 0]
def main():
    node = 'news'   # 크롤링 할 대상
    srcText = '플랫폼'
    cnt = 0
    result = []
    jsonResult = []

    jsonResponse= getNaverSearch(node, srcText, 1, 100)  #[CODE 2]
    total = jsonResponse['total']
 
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):         
        for post in jsonResponse['items']:
            getPostData(post, jsonResult, cnt)  #[CODE 3]
        
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)  #[CODE 2]
       
    print('전체 검색 : %d 건' %total)
    
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent=4, sort_keys=True,  ensure_ascii=False)
                        
        outfile.write(jsonFile)
        
    columns = ["cnt","제목","요약","org_link","pDate"]
    result_df = pd.DataFrame(result, columns = columns)
    result_df.to_csv('./양현직-201835692-1번.csv',
                index=False, encoding='cp949')  
    
    print ('%s_naver_%s.json SAVED' % (srcText, node))
    
if __name__ == '__main__':
    main()



[2023-06-13 11:15:30.973151] Url Request Success
[2023-06-13 11:15:31.088886] Url Request Success
[2023-06-13 11:15:31.201338] Url Request Success
[2023-06-13 11:15:31.405698] Url Request Success
[2023-06-13 11:15:31.533827] Url Request Success
[2023-06-13 11:15:31.681914] Url Request Success
[2023-06-13 11:15:31.801962] Url Request Success
[2023-06-13 11:15:32.050693] Url Request Success
[2023-06-13 11:15:32.192041] Url Request Success
[2023-06-13 11:15:32.391895] Url Request Success
HTTP Error 400: Bad Request
[2023-06-13 11:15:32.498268] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%ED%94%8C%EB%9E%AB%ED%8F%BC&start=1001&display=100
전체 검색 : 4765325 건
플랫폼_naver_news.json SAVED


In [29]:
conda install -c conda-forge jpype

Retrieving notices: ...working... done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - jpype

Current channels:

  - https://conda.anaconda.org/conda-forge/osx-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import glob
import re
from functools import reduce

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from collections import Counter

import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

In [ ]:
import glob

all_files=glob.glob('./양현직-201835692-1번.csv')
all_files

In [ ]:
all_title = all_files['제목']
all_title

In [ ]:
import re

message = ''  # 누적할 메시지 변수 초기화

for item in all_title:
    message = message + re.sub(r'[^\w]', ' ', str(item)) + ''

print(message)  # 메시지 값 출력

In [ ]:
import json
import re

from konlpy.tag import Okt

from collections import Counter

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from wordcloud import WordCloud

In [ ]:
nlp = Okt()
message_N = nlp.nouns(message)
message_N   #출력하여 내용 확인

In [ ]:
count = Counter(message_N)

count   #출력하여 내용 확인

In [ ]:
word_count = dict()

for tag, counts in count.most_common(80):
    if(len(str(tag))>1):
        word_count[tag] = counts
        print("%s : %d" % (tag, counts))

In [ ]:
font_path = "/System/Library/Fonts/AppleSDGothicNeo.ttc"
font_name = font_manager.FontProperties(fname = font_path).get_name()
matplotlib.rc('font', family=font_name)

In [ ]:
plt.figure(figsize=(12,5))
plt.xlabel('키워드')
plt.ylabel('빈도수')
plt.grid(True)

sorted_Keys = sorted(word_count, key=word_count.get, reverse=True)
sorted_Values = sorted(word_count.values(), reverse=True)

plt.bar(range(len(word_count)), sorted_Values, align='center')
plt.xticks(range(len(word_count)), list(sorted_Keys), rotation='75')

plt.show()

In [ ]:
wc = WordCloud(font_path, background_color='ivory', width=800, height=600)
cloud=wc.generate_from_frequencies(word_count)

plt.figure(figsize=(8,8))
plt.imshow(cloud)
plt.axis('off')
plt.show()